In [1]:
from operator import add
import re
from collections import OrderedDict
from operator import itemgetter 
import itertools
from pyspark.sql import SparkSession

spark_session = SparkSession\
       .builder\
       .master("spark://192.168.1.153:7077") \
       .appName("jiayi_a")\
       .config("spark.dynamicAllocation.enabled", True)\
       .config("spark.shuffle.service.enabled", True)\
       .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
       .config("spark.executor.cores",4)\
       .getOrCreate()
        
spark_context = spark_session.sparkContext

In [2]:
# Question A.1

# A.1.1
rdd_en = spark_context.textFile('hdfs://192.168.1.153:9000/europarl/europarl-v7.sv-en.en').cache()
lcount_en = rdd_en.count()
print("number of lines in English:",lcount_en)

# A.1.2
rdd_sv = spark_context.textFile('hdfs://192.168.1.153:9000/europarl/europarl-v7.sv-en.sv').cache()
lcount_sv = rdd_sv.count()
print("number of lines in Swedish:",lcount_sv)

# A.1.3
if lcount_en - lcount_sv == 0:
    print("same")

# A.1.4
print("number of partitions:", rdd_en.getNumPartitions()+rdd_sv.getNumPartitions())

number of lines in English: 1862234
number of lines in Swedish: 1862234
same
number of partitions: 5


In [3]:
# Question A.2

def preprocess(rdd):return rdd.map(lambda line: line.lower()).map(lambda words: words.split(' '))

# A.2.1
rdd_en = preprocess(rdd_en)
rdd_sv = preprocess(rdd_sv)

#A.2.2
print(rdd_en.take(10))
print(rdd_sv.take(10))

#A.2.3
if rdd_en.count()-rdd_sv.count() == 0:
    print("same")

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf', 'of', '

In [4]:
# Question A.3

# A.3.1
word_en = rdd_en.flatMap(lambda x: x).map(lambda w: w.strip()).map(lambda w:(w,1)).reduceByKey(add).takeOrdered(10, key=lambda x: -x[1])
word_sv = rdd_sv.flatMap(lambda x: x).map(lambda w: w.strip()).map(lambda w:(w,1)).reduceByKey(add).takeOrdered(10, key=lambda x: -x[1])

print(word_en)
print(word_sv)

[('the', 3498452), ('of', 1659758), ('to', 1539760), ('and', 1288402), ('in', 1085994), ('that', 797519), ('a', 773522), ('is', 758050), ('for', 534242), ('we', 522851)]
[('att', 1706293), ('och', 1344830), ('i', 1050774), ('det', 924868), ('som', 913276), ('för', 908680), ('av', 738068), ('är', 694381), ('en', 620310), ('vi', 539797)]


In [4]:
# Question A.4

en_1 = rdd_en.zipWithIndex()
sv_1 = rdd_sv.zipWithIndex()

en_2 = en_1.map(lambda x: (x[1], x[0]))
sv_2 = sv_1.map(lambda x: (x[1], x[0]))

en_sv_3 = en_2.join(sv_2)

en_sv_4 = en_sv_3.filter(lambda x: [''] not in x[1])

en_sv_5 = en_sv_4.filter(lambda x: len(x[1][0])<20 and len(x[1][1])<20)  # leave sentences with under 20 words

en_sv_6 = en_sv_5.filter(lambda x: len(x[1][0])==len(x[1][1]))

en_sv_7 = en_sv_6.flatMap(lambda x: list(zip(x[1][0],x[1][1])))

en_sv_8 = en_sv_7.map(lambda w:(w,1)).reduceByKey(add)

# step 9
print(en_sv_8.takeOrdered(20, key=lambda x: -x[1]))

[(('is', 'är'), 19561), (('we', 'vi'), 15616), (('i', 'jag'), 15265), (('and', 'och'), 15119), (('in', 'i'), 8768), (('a', 'en'), 8676), (('it', 'det'), 8506), (('to', 'att'), 7691), (('this', 'detta'), 7645), (('that', 'att'), 6775), (('not', 'inte'), 6748), (('the', 'den'), 5915), (('of', 'av'), 5744), (('a', 'ett'), 5384), (('for', 'för'), 5289), (('have', 'har'), 5004), (('the', 'det'), 4871), (('that', 'det'), 4588), (('the', 'de'), 4148), (('are', 'är'), 3914)]
